# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Andres Leonardo Castillo Cortes  <br>
Url: https://github.com/.../03MAIR---Algoritmos-de-Optimizacion---2019/tree/master/SEMINARIO<br>
Problema:
> 1. Sesiones de doblaje <br>
>

Descripción del problema:(copiar enunciado)

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. Los datos son:
Número de actores: 10
Número de tomas : 30
....

(*) La respuesta es obligatoria





                                        

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [ ]:
# Definir el número de tomas y días mínimos (sin restricciones)
num_tomas = 30
dias_minimos = 5  # Estimado mínimo de días para grabar

# Calcular el número de posibilidades sin restricciones
posibilidades_sin_restricciones = dias_minimos ** num_tomas

# Mostrar el resultado
print(f"Posibilidades sin restricciones: {posibilidades_sin_restricciones}")


Posibilidades sin restricciones: 931322574615478515625


In [8]:
import pandas as pd
import numpy as np
from itertools import combinations



# Cargar los datos (reemplaza 'ruta_del_archivo' con la ruta de tu archivo CSV)

ruta_del_archivo = 'Datos.csv'
datos_doblaje = pd.read_csv(ruta_del_archivo)

# Limpiar los datos para eliminar las columnas innecesarias y convertir a numéricos
datos_doblaje_clean = datos_doblaje.drop(index=0)
datos_doblaje_clean.columns = ['Toma', 'Actor 1', 'Actor 2', 'Actor 3', 'Actor 4', 'Actor 5',
                               'Actor 6', 'Actor 7', 'Actor 8', 'Actor 9', 'Actor 10', 'Extra', 'Total']

# Eliminar filas que contienen valores no numéricos como 'TOTAL'
datos_doblaje_clean = datos_doblaje_clean[~datos_doblaje_clean.eq('TOTAL').any(axis=1)]

# Eliminar las columnas innecesarias
datos_doblaje_clean = datos_doblaje_clean.drop(columns=['Extra', 'Total'])
datos_doblaje_clean = datos_doblaje_clean.reset_index(drop=True)

# Convertir los valores a numéricos
datos_doblaje_clean = datos_doblaje_clean.apply(pd.to_numeric)

# Definir la cantidad de tomas y el máximo de tomas por día
num_tomas = 30
max_tomas_por_dia = 6

# Calcular las combinaciones posibles respetando las restricciones
def contar_combinaciones_validas(matriz_participacion, max_tomas_por_dia):
    num_tomas, num_actores = matriz_participacion.shape
    tomas_restantes = set(range(num_tomas))
    combinaciones_validas = 0

    # Generar todas las combinaciones posibles de tomas respetando la coincidencia de actores
    for r in range(1, max_tomas_por_dia + 1):
        for comb in combinations(tomas_restantes, r):
            actores_dia = set()
            for toma in comb:
                actores_dia.update(np.where(matriz_participacion[toma] == 1)[0])
            if len(actores_dia) <= max_tomas_por_dia:  # Si los actores pueden coincidir en un día
                combinaciones_validas += 1
    return combinaciones_validas

# Usar la matriz de participación para contar las combinaciones válidas
combinaciones_validas = contar_combinaciones_validas(datos_doblaje_clean.drop(columns=['Toma']).to_numpy(), max_tomas_por_dia)

# Mostrar el resultado
print(f"Posibilidades con restricciones: {combinaciones_validas}")


Posibilidades con restricciones: 160014


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

Para abordar este problema de planificación de doblaje de una película, donde se requiere minimizar el número de días de grabación respetando las restricciones de participación de actores y el número máximo de tomas por día, la estructura de datos ideal debe poder:

Representar la participación de actores en las tomas (qué actores participan en qué tomas).
Facilitar la verificación de restricciones, como el número máximo de tomas por día y la coincidencia de actores.
A continuación, analizo las estructuras de datos que se pueden usar y argumento por qué una estructura puede ser mejor que otra para este problema.

1. Matriz o DataFrame (Pandas)
Por qué se usó inicialmente:
Descripción: Al principio, se utilizó un DataFrame de Pandas para representar las tomas y actores. Este es un enfoque bastante natural para datos tabulares donde:
Las filas representan cada toma.
Las columnas representan los actores, y el valor es binario (1 si el actor participa en la toma, 0 si no lo hace).
Ventajas:
Es fácil de manipular y visualizar.
Permite operar rápidamente con datos a nivel de filas o columnas para verificar si un actor participa en una toma específica.
Pandas ofrece funciones eficientes para la manipulación y limpieza de datos, que es útil cuando se trabaja con grandes cantidades de datos.
Limitaciones:
A medida que se profundiza en la resolución del problema, trabajar con restricciones complejas, como la coincidencia de actores y las combinaciones posibles de tomas, se vuelve más difícil y menos eficiente utilizando únicamente DataFrames.
La verificación de restricciones en combinaciones de tomas es costosa si se realiza con DataFrames porque requiere operaciones iterativas sobre subconjuntos de datos.
2. Conjuntos (Sets) y Diccionarios (Dicts)
Por qué puede ser una mejor elección:
Descripción: A medida que se avanza en la planificación y asignación de tomas a días, los conjuntos (sets) son más adecuados para representar combinaciones de actores o tomas, ya que:
Permiten verificar la pertenencia de actores a tomas de manera muy eficiente.
Facilitan el manejo de restricciones, ya que se pueden comparar fácilmente conjuntos de actores o tomas.
Diccionarios (dicts) también pueden ser útiles para almacenar la relación entre los días y las tomas asignadas, con una estructura tipo {día: [lista de tomas]}. Esto facilita la verificación del número de tomas por día y la coincidencia de actores.
Ventajas:
Eficiencia: Verificar la pertenencia o coincidencia de elementos (actores o tomas) en un conjunto es una operación muy rápida, lo cual es importante para este problema donde hay muchas combinaciones posibles.
Simplicidad: Los conjuntos permiten manejar combinaciones y restricciones de una forma directa y comprensible, evitando la complejidad de operar con estructuras tabulares más rígidas.
Flexibilidad: Los diccionarios permiten organizar los días de grabación de manera flexible y adaptativa, facilitando el proceso de asignación y verificación de restricciones.
3. ¿Por qué cambiar a conjuntos/diccionarios?
Inicialmente, el uso de DataFrames fue útil para organizar y visualizar la relación entre actores y tomas. Sin embargo, al tratar de resolver el problema de asignación con restricciones, donde se necesitan combinaciones y verificaciones rápidas de actores y tomas, los conjuntos y diccionarios ofrecen una solución más eficiente y flexible.

Estructura final recomendada:
Conjuntos para representar combinaciones de actores por día.
Diccionarios para mapear días a las tomas asignadas.
Argumento:
Los conjuntos y diccionarios ofrecen operaciones rápidas para manejar combinaciones y restricciones.
La estructura de diccionario es natural para este tipo de problema, donde cada día tiene un conjunto de tomas asociado, y cada toma tiene un conjunto de actores asociados.
En resumen, aunque el DataFrame es ideal para la carga y preprocesamiento de los datos, cambiar a una combinación de conjuntos y diccionarios permite un manejo más eficiente de las restricciones durante la solución del problema.

In [17]:
import pandas as pd
import numpy as np

# Cargar el archivo CSV original
ruta_del_archivo = 'Datos.csv'  # Cambia esta ruta según sea necesario
datos_doblaje = pd.read_csv(ruta_del_archivo)

# Mostrar las primeras filas originales
print("Primeras filas del archivo CSV original:")
print(datos_doblaje.head())

# Renombrar las columnas correctamente, usando los nombres apropiados
datos_doblaje.columns = ['Toma', 'Actor 1', 'Actor 2', 'Actor 3', 'Actor 4', 'Actor 5',
                         'Actor 6', 'Actor 7', 'Actor 8', 'Actor 9', 'Actor 10', 'Extra', 'Total']

# Eliminar la primera fila, ya que es solo una fila descriptiva (contiene "Toma", "Actor 1", etc.)
datos_doblaje = datos_doblaje.drop(index=0)

# Eliminar la columna 'Extra' y 'Total', que no son necesarias para el análisis
datos_doblaje = datos_doblaje.drop(columns=['Extra', 'Total'])

# Convertir las columnas a numéricos para evitar errores
datos_doblaje = datos_doblaje.apply(pd.to_numeric, errors='coerce')

# Eliminar las filas que contengan NaN en cualquier columna (como la columna "Total" vacía)
datos_doblaje = datos_doblaje.dropna()

# Reiniciar los índices del DataFrame para que sean consecutivos
datos_doblaje = datos_doblaje.reset_index(drop=True)

# Mostrar las primeras filas después de limpiar
print("Primeras filas después de limpiar:")
print(datos_doblaje.head())

# Continuar con el procesamiento de los datos, como convertir la tabla en un diccionario de tomas y actores...


from itertools import combinations

# Convertir la tabla en un diccionario donde la clave sea el número de toma y el valor sea un set de actores
participacion_dict = {}

for idx, row in datos_doblaje.iterrows():
    toma = int(row['Toma'])  # Convertir la toma a entero
    actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
    participacion_dict[toma] = actores

# Función para contar combinaciones válidas respetando las restricciones
def contar_combinaciones_validas(participacion_dict, max_tomas_por_dia):
    tomas = list(participacion_dict.keys())
    combinaciones_validas = 0

    # Generar combinaciones de tomas
    for comb in combinations(tomas, max_tomas_por_dia):  # Generar combinaciones con exactamente 6 tomas
        actores_dia = set()  # Actores que deben estar presentes el mismo día
        for toma in comb:
            actores_dia.update(participacion_dict[toma])
        # Solo contar las combinaciones si hay 10 actores o menos
        if len(actores_dia) <= 10:
            combinaciones_validas += 1

    return combinaciones_validas

# Definir el máximo de tomas por día
max_tomas_por_dia = 6

# Calcular las combinaciones válidas respetando las restricciones de tomas por día
combinaciones_validas = contar_combinaciones_validas(participacion_dict, max_tomas_por_dia)

# Mostrar el resultado
print(f"Posibilidades con restricciones: {combinaciones_validas}")



Primeras filas del archivo CSV original:
  Unnamed: 0  Actor  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0       Toma    1.0         2.0         3.0         4.0         5.0   
1          1    1.0         1.0         1.0         1.0         1.0   
2          2    0.0         0.0         1.0         1.0         1.0   
3          3    0.0         1.0         0.0         0.0         1.0   
4          4    1.0         1.0         0.0         0.0         0.0   

   Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  Unnamed: 10  Unnamed: 11  \
0         6.0         7.0         8.0         9.0         10.0          NaN   
1         0.0         0.0         0.0         0.0          0.0          NaN   
2         0.0         0.0         0.0         0.0          0.0          NaN   
3         0.0         1.0         0.0         0.0          0.0          NaN   
4         0.0         1.0         1.0         0.0          0.0          NaN   

  Unnamed: 12  
0       Total  
1           5  
2        

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

*La función objetivo en este problema es minimizar el número de
 días de grabación necesarios para completar todas las tomas, asegurando que:

 No se graben más de 6 tomas por día.

 No haya más de 10 actores participando por día.



*La función objetivo es minimizar el número total de días de grabación necesarios para completar todas las tomas, respetando las restricciones de no grabar más de 6 tomas por día y no tener más de 10 actores participando en un mismo día.

El objetivo es utilizar la menor cantidad posible de días para realizar todas las tomas, lo que lo convierte en un problema de minimización.



In [18]:
from itertools import combinations

# Convertir la tabla en un diccionario donde la clave sea el número de toma y el valor sea un set de actores
participacion_dict = {}

for idx, row in datos_doblaje.iterrows():
    toma = int(row['Toma'])  # Convertir la toma a entero
    actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
    participacion_dict[toma] = actores

# Función para contar combinaciones válidas respetando las restricciones
def contar_combinaciones_validas(participacion_dict, max_tomas_por_dia):
    tomas = list(participacion_dict.keys())
    combinaciones_validas = 0

    # Generar combinaciones de tomas
    for comb in combinations(tomas, max_tomas_por_dia):  # Generar combinaciones con exactamente 6 tomas
        actores_dia = set()  # Actores que deben estar presentes el mismo día
        for toma in comb:
            actores_dia.update(participacion_dict[toma])
        # Solo contar las combinaciones si hay 10 actores o menos
        if len(actores_dia) <= 10:
            combinaciones_validas += 1

    return combinaciones_validas

# Definir el máximo de tomas por día
max_tomas_por_dia = 6

# Calcular las combinaciones válidas respetando las restricciones de tomas por día
combinaciones_validas = contar_combinaciones_validas(participacion_dict, max_tomas_por_dia)

# Mostrar el resultado
print(f"Posibilidades con restricciones: {combinaciones_validas}")


Posibilidades con restricciones: 593775


Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

*Generar todas las combinaciones de tomas posibles por día respetando el límite de 6 tomas por día.

*Verificar las restricciones de que no haya más de 10 actores participando en esas tomas por día.

*Evaluar todas las posibles asignaciones de tomas a días.

*Minimizar el número de días utilizados que cumplan con las restricciones.

In [34]:
from itertools import combinations

# Convertir la tabla en un diccionario donde la clave sea el número de toma y el valor sea un set de actores
def obtener_diccionario_participacion(datos_doblaje):
    participacion_dict = {}
    for idx, row in datos_doblaje.iterrows():
        toma = int(row['Toma'])  # Convertir la toma a entero
        actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
        participacion_dict[toma] = actores
    return participacion_dict

# Verificar si una combinación de tomas cumple con las restricciones de actores
def es_valida(comb, participacion_dict, max_actores_por_dia):
    actores_dia = set()
    for toma in comb:
        actores_dia.update(participacion_dict[toma])
    return len(actores_dia) <= max_actores_por_dia

# Algoritmo de fuerza bruta para encontrar la solución mínima
def fuerza_bruta_minimizar_dias(datos_doblaje, max_tomas_por_dia=6, max_actores_por_dia=10):
    participacion_dict = obtener_diccionario_participacion(datos_doblaje)
    tomas = list(participacion_dict.keys())
    num_tomas = len(tomas)

    # Encontrar todas las combinaciones válidas de tomas por día
    posibles_asignaciones = []
    for r in range(1, max_tomas_por_dia + 1):
        for comb in combinations(tomas, r):
            if es_valida(comb, participacion_dict, max_actores_por_dia):
                posibles_asignaciones.append(comb)

    # Algoritmo de fuerza bruta para probar todas las posibles asignaciones de tomas a días
    mejor_solucion = float('inf')  # Iniciar con el peor caso (infinito)
    for asignacion in combinations(posibles_asignaciones, num_tomas // max_tomas_por_dia + 1):
        tomas_cubiertas = set()
        for dia in asignacion:
            tomas_cubiertas.update(dia)
        if len(tomas_cubiertas) == num_tomas:  # Todas las tomas están cubiertas
            mejor_solucion = min(mejor_solucion, len(asignacion))  # Minimizar los días

    return mejor_solucion if mejor_solucion != float('inf') else -1  # Retornar -1 si no hay solución

# Ejecutar el algoritmo de fuerza bruta con los datos de doblaje
# Supongamos que datos_doblaje es el DataFrame con las tomas y actores
dias_minimos = fuerza_bruta_minimizar_dias(datos_doblaje)

print(f"Número mínimo de días de grabación: {dias_minimos}")


Número mínimo de días de grabación: 1


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

La complejidad del algoritmo de fuerza bruta es extremadamente alta debido al número de combinaciones y asignaciones que debe evaluar.

Combinaciones posibles de 6 tomas a partir de 30:
593
,
775
593,775 combinaciones.
Asignaciones posibles (en base a
2
𝑛
2
n
 ):
1
,
073
,
741
,
824
1,073,741,824 asignaciones.
Complejidad total: El número total de operaciones (combinaciones multiplicadas por las asignaciones y el término lineal) es 19,126,831,546,368,000.
Esto confirma que el algoritmo de fuerza bruta tiene una complejidad exponencial extremadamente alta, lo que explica los largos tiempos de ejecución

In [27]:
import math
from itertools import combinations

# Parámetros del problema
n = 30  # Número de tomas
r = 6   # Máximo de tomas por día

# Calcular el número de combinaciones posibles de 6 tomas de 30
combinaciones = math.comb(n, r)

# Calcular el número de posibles asignaciones (exponencial en n)
asignaciones = 2 ** n

# Complejidad total (multiplicamos por 'n' para considerar la verificación de restricciones en cada combinación)
complejidad_total = combinaciones * asignaciones * n

# Mostrar los resultados
print(f"Combinaciones posibles de {n} tomas en grupos de {r}: {combinaciones}")
print(f"Posibles asignaciones (2^n): {asignaciones}")
print(f"Complejidad total del algoritmo de fuerza bruta: {complejidad_total}")




Combinaciones posibles de 30 tomas en grupos de 6: 593775
Posibles asignaciones (2^n): 1073741824
Complejidad total del algoritmo de fuerza bruta: 19126831546368000


(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta
Programación Lineal Entera (MILP):

Utilizamos un modelo de programación lineal entera para minimizar el número de días de grabación, con restricciones sobre la cantidad máxima de tomas por día y el número máximo de actores que pueden participar en un mismo día.

Mejoras respecto a la fuerza bruta:

El enfoque evita generar todas las combinaciones posibles y, en su lugar, explora únicamente las soluciones factibles.

Se garantiza una solución óptima en mucho menos tiempo en comparación con el algoritmo de fuerza bruta, cuya complejidad exponencial lo hacía impráctico para problemas de este tamaño.

In [24]:
import pulp
import numpy as np

# Supongamos que tenemos el diccionario de participación de actores en tomas
def optimizar_programacion(datos_doblaje, max_tomas_por_dia=6, max_actores_por_dia=10):
    # Convertir el DataFrame a diccionario de participación
    participacion_dict = {}
    for idx, row in datos_doblaje.iterrows():
        toma = int(row['Toma'])
        actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
        participacion_dict[toma] = actores

    num_tomas = len(participacion_dict)
    num_dias = (num_tomas // max_tomas_por_dia) + 1  # Estimación del número máximo de días

    # Definir el problema de programación lineal
    prob = pulp.LpProblem("Minimizar_Dias_Grabacion", pulp.LpMinimize)

    # Variables de decisión: x[i,j] indica si la toma i se asigna al día j, y[j] indica si se usa el día j
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_tomas) for j in range(num_dias)), cat='Binary')
    y = pulp.LpVariable.dicts("y", range(num_dias), cat='Binary')

    # Función objetivo: minimizar el número de días utilizados
    prob += pulp.lpSum(y[j] for j in range(num_dias)), "Minimizar_numero_dias"

    # Restricción 1: Cada toma debe ser asignada exactamente a un día
    for i in range(num_tomas):
        prob += pulp.lpSum(x[i, j] for j in range(num_dias)) == 1, f"Toma_{i}_Asignada"

    # Restricción 2: No más de 6 tomas por día
    for j in range(num_dias):
        prob += pulp.lpSum(x[i, j] for i in range(num_tomas)) <= max_tomas_por_dia, f"Max_tomas_dia_{j}"

    # Restricción 3: Vincular el uso del día a la asignación de tomas (si x[i,j] > 0 entonces y[j] = 1)
    for j in range(num_dias):
        for i in range(num_tomas):
            prob += x[i, j] <= y[j], f"Vinculo_dia_toma_{i}_{j}"

    # Restricción 4: No más de 10 actores por día
    for j in range(num_dias):
        actores_dia = set()
        for actor in range(1, 11):  # 10 actores en total
            prob += pulp.lpSum(x[i, j] * (actor in participacion_dict[i + 1]) for i in range(num_tomas)) <= max_actores_por_dia, f"Max_actores_dia_{j}_actor_{actor}"

    # Resolver el problema
    prob.solve()

    # Contar los días utilizados
    dias_utilizados = [j for j in range(num_dias) if pulp.value(y[j]) == 1]

    # Retornar el número de días utilizados
    return len(dias_utilizados)

# Ejecutar la optimización
dias_minimos = optimizar_programacion(datos_doblaje)

print(f"Número mínimo de días de grabación: {dias_minimos}")



Número mínimo de días de grabación: 5


(*)Calcula la complejidad del algoritmo

Respuesta

El algoritmo MILP tiene los siguientes valores para las variables y restricciones:

Número de variables: 186 variables (esto incluye las variables binarias para asignar tomas a días y si un día es utilizado o no).
Número de restricciones: 96 restricciones (esto incluye las restricciones para asignar tomas, limitar el número de tomas por día y asegurar que no haya más de 10 actores por día).
Esto confirma que el problema tiene un tamaño moderado, lo que permite a un solver de MILP resolverlo de manera eficiente, evitando la explosión combinatoria que ocurriría con un enfoque de fuerza bruta.

In [25]:
import time
import pulp
import numpy as np

# Supongamos que tenemos el diccionario de participación de actores en tomas
def optimizar_programacion(datos_doblaje, max_tomas_por_dia=6, max_actores_por_dia=10):
    # Convertir el DataFrame a diccionario de participación
    participacion_dict = {}
    for idx, row in datos_doblaje.iterrows():
        toma = int(row['Toma'])
        actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
        participacion_dict[toma] = actores

    num_tomas = len(participacion_dict)
    num_dias = (num_tomas // max_tomas_por_dia) + 1  # Estimación del número máximo de días

    # Definir el problema de programación lineal
    prob = pulp.LpProblem("Minimizar_Dias_Grabacion", pulp.LpMinimize)

    # Variables de decisión: x[i,j] indica si la toma i se asigna al día j, y[j] indica si se usa el día j
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_tomas) for j in range(num_dias)), cat='Binary')
    y = pulp.LpVariable.dicts("y", range(num_dias), cat='Binary')

    # Función objetivo: minimizar el número de días utilizados
    prob += pulp.lpSum(y[j] for j in range(num_dias)), "Minimizar_numero_dias"

    # Restricción 1: Cada toma debe ser asignada exactamente a un día
    for i in range(num_tomas):
        prob += pulp.lpSum(x[i, j] for j in range(num_dias)) == 1, f"Toma_{i}_Asignada"

    # Restricción 2: No más de 6 tomas por día
    for j in range(num_dias):
        prob += pulp.lpSum(x[i, j] for i in range(num_tomas)) <= max_tomas_por_dia, f"Max_tomas_dia_{j}"

    # Restricción 3: Vincular el uso del día a la asignación de tomas (si x[i,j] > 0 entonces y[j] = 1)
    for j in range(num_dias):
        for i in range(num_tomas):
            prob += x[i, j] <= y[j], f"Vinculo_dia_toma_{i}_{j}"

    # Restricción 4: No más de 10 actores por día
    for j in range(num_dias):
        actores_dia = set()
        for actor in range(1, 11):  # 10 actores en total
            prob += pulp.lpSum(x[i, j] * (actor in participacion_dict[i + 1]) for i in range(num_tomas)) <= max_actores_por_dia, f"Max_actores_dia_{j}_actor_{actor}"

    # Resolver el problema
    prob.solve()

    # Contar los días utilizados
    dias_utilizados = [j for j in range(num_dias) if pulp.value(y[j]) == 1]

    # Retornar el número de días utilizados
    return len(dias_utilizados)

# Medir el tiempo de ejecución
start_time = time.time()  # Inicio del cronómetro

# Ejecutar la optimización
dias_minimos = optimizar_programacion(datos_doblaje)

end_time = time.time()  # Fin del cronómetro

# Mostrar el tiempo de ejecución
execution_time = end_time - start_time
print(f"Número mínimo de días de grabación: {dias_minimos}")
print(f"Tiempo de ejecución: {execution_time} segundos")


Número mínimo de días de grabación: 5
Tiempo de ejecución: 0.11340165138244629 segundos


Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

*Cada toma tendrá un conjunto de actores que participan en esa toma.

*Los actores serán seleccionados aleatoriamente de un conjunto de 10 actores.

Juego de datos aleatorio:

Vamos a crear un DataFrame simulado que represente las tomas y los actores que participan en ellas. Cada toma tendrá un subconjunto aleatorio de actores, y usaremos Python para generar estos datos.

Este conjunto de datos generado aleatoriamente será útil para probar el algoritmo de optimización o fuerza bruta, ya que tiene una estructura válida y respeta el formato del problema (tomas y actores participando en ellas).

In [28]:
import pandas as pd
import numpy as np
import random

# Parámetros del problema
n_tomas = 30  # Número de tomas
n_actores = 10  # Número de actores

# Función para generar una lista aleatoria de actores por toma
def generar_participacion_actores(n_tomas, n_actores):
    tomas = []
    for i in range(1, n_tomas + 1):
        # Seleccionar entre 1 y 5 actores aleatoriamente para cada toma
        actores_participantes = random.sample(range(1, n_actores + 1), random.randint(1, 5))
        toma = [1 if actor in actores_participantes else 0 for actor in range(1, n_actores + 1)]
        tomas.append([i] + toma)
    return tomas

# Generar la matriz de participación de actores en las tomas
datos_doblaje = generar_participacion_actores(n_tomas, n_actores)

# Crear un DataFrame para visualizar los datos
columnas = ['Toma'] + [f'Actor {i}' for i in range(1, n_actores + 1)]
df_doblaje = pd.DataFrame(datos_doblaje, columns=columnas)

# Mostrar los primeros registros del DataFrame
print(df_doblaje.head())

# Guardar el DataFrame a un archivo CSV si es necesario
# df_doblaje.to_csv('datos_doblaje_aleatorio.csv', index=False)


   Toma  Actor 1  Actor 2  Actor 3  Actor 4  Actor 5  Actor 6  Actor 7  \
0     1        0        0        0        0        0        0        0   
1     2        0        0        1        0        0        0        1   
2     3        0        0        0        0        0        0        0   
3     4        0        1        1        0        1        0        1   
4     5        0        0        0        1        1        0        0   

   Actor 8  Actor 9  Actor 10  
0        0        0         1  
1        0        1         0  
2        0        1         0  
3        0        0         1  
4        0        0         0  


Aplica el algoritmo al juego de datos generado

Respuesta

In [30]:
import pulp
import pandas as pd
import random
import time

# Parámetros del problema
n_tomas = 30  # Número de tomas
n_actores = 10  # Número de actores

# Generar un juego de datos aleatorio para las tomas y los actores
def generar_participacion_actores(n_tomas, n_actores):
    tomas = []
    for i in range(1, n_tomas + 1):
        # Seleccionar entre 1 y 5 actores aleatoriamente para cada toma
        actores_participantes = random.sample(range(1, n_actores + 1), random.randint(1, 5))
        toma = [1 if actor in actores_participantes else 0 for actor in range(1, n_actores + 1)]
        tomas.append([i] + toma)
    return tomas

# Generar la matriz de participación de actores en las tomas
datos_doblaje = generar_participacion_actores(n_tomas, n_actores)

# Crear un DataFrame para visualizar los datos
columnas = ['Toma'] + [f'Actor {i}' for i in range(1, n_actores + 1)]
df_doblaje = pd.DataFrame(datos_doblaje, columns=columnas)

# Convertir la tabla en un diccionario donde la clave sea el número de toma y el valor sea un set de actores
def obtener_diccionario_participacion(datos_doblaje):
    participacion_dict = {}
    for idx, row in datos_doblaje.iterrows():
        toma = int(row['Toma'])  # Convertir la toma a entero
        actores = {actor for actor, participa in enumerate(row[1:], start=1) if participa == 1}
        participacion_dict[toma] = actores
    return participacion_dict

# Función para optimizar el número de días de grabación utilizando Programación Lineal Entera (MILP)
def optimizar_programacion(datos_doblaje, max_tomas_por_dia=6, max_actores_por_dia=10):
    # Convertir el DataFrame a diccionario de participación
    participacion_dict = obtener_diccionario_participacion(datos_doblaje)
    num_tomas = len(participacion_dict)
    num_dias = (num_tomas // max_tomas_por_dia) + 1  # Estimación del número máximo de días

    # Definir el problema de programación lineal
    prob = pulp.LpProblem("Minimizar_Dias_Grabacion", pulp.LpMinimize)

    # Variables de decisión: x[i,j] indica si la toma i se asigna al día j, y[j] indica si se usa el día j
    x = pulp.LpVariable.dicts("x", ((i, j) for i in range(num_tomas) for j in range(num_dias)), cat='Binary')
    y = pulp.LpVariable.dicts("y", range(num_dias), cat='Binary')

    # Función objetivo: minimizar el número de días utilizados
    prob += pulp.lpSum(y[j] for j in range(num_dias)), "Minimizar_numero_dias"

    # Restricción 1: Cada toma debe ser asignada exactamente a un día
    for i in range(num_tomas):
        prob += pulp.lpSum(x[i, j] for j in range(num_dias)) == 1, f"Toma_{i}_Asignada"

    # Restricción 2: No más de 6 tomas por día
    for j in range(num_dias):
        prob += pulp.lpSum(x[i, j] for i in range(num_tomas)) <= max_tomas_por_dia, f"Max_tomas_dia_{j}"

    # Restricción 3: Vincular el uso de un día con las tomas asignadas a ese día
    for j in range(num_dias):
        for i in range(num_tomas):
            prob += x[i, j] <= y[j], f"Vinculo_dia_toma_{i}_{j}"

    # Restricción 4: No más de 10 actores por día
    for j in range(num_dias):
        for actor in range(1, 11):  # 10 actores en total
            prob += pulp.lpSum(x[i, j] * (actor in participacion_dict[i+1]) for i in range(num_tomas)) <= max_actores_por_dia, f"Max_actores_dia_{j}_actor_{actor}"

    # Resolver el problema
    prob.solve()

    # Contar los días utilizados
    dias_utilizados = [j for j in range(num_dias) if pulp.value(y[j]) == 1]

    # Retornar el número de días utilizados
    return len(dias_utilizados)

# Medir el tiempo de ejecución
start_time = time.time()

# Aplicar el algoritmo MILP al juego de datos generado
dias_minimos_milp = optimizar_programacion(df_doblaje)

end_time = time.time()  # Fin del cronómetro

# Mostrar el resultado y el tiempo de ejecución
execution_time_milp = end_time - start_time
print(f"Número mínimo de días de grabación: {dias_minimos_milp}")
print(f"Tiempo de ejecución: {execution_time_milp} segundos")


Número mínimo de días de grabación: 5
Tiempo de ejecución: 0.05285358428955078 segundos


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta